In [1]:
from google.colab import drive

# mount drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Adapted from the following tutorial: https://colab.research.google.com/github/csaybar/EEwPython/blob/master/cnn_demo.ipynb#scrollTo=cTcFXVBRQ8RN

Main steps for building the U-Net:
1. Create the dataset comprising 256x256 pixel patches.
2. Visualize data/perform some exploratory data analysis.
3. Set up data pipeline and complete preprocessing.
4. Initialize the model's parameters.
5. Loop:
  - Calculate current loss (forward propagation)
  - Calculate current gradient (backward propagation)
  - Update parameters (gradient descent)

====================================================================================

### Installation
Install required packages and extensions.

In [ ]:
!pip install tensorflow                 # tensorflow
!pip install earthengine-api            # earthengine API
!pip install --upgrade earthengine-api

# load the TensorBoard notebook extension
%load_ext tensorboard

### 2. Authentication
Authenticate with required services.

In [2]:
# Google Colab
from google.colab import auth
auth.authenticate_user()

In [4]:
# Google Earth Engine
!earthengine authenticate --force

W0318 09:14:40.472917 132048745291776 _default.py:683] No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable


Initialize and test software set-up.

In [6]:
# Earth Engine Python API
import ee
print('Earth Engine version: ' + ee.__version__)
# ee.Initialize(project='gbsc-gcp-lab-emordeca')

# Tensorflow
import tensorflow as tf
print('Tensorflow version: ' + tf.__version__)

# Folium
import folium
print('Folium version: ' + folium.__version__)

# define the URL format used for Earth Engine generated map tiles
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

Earth Engine version: 0.1.394
Tensorflow version: 2.15.0
Folium version: 0.14.0


### Prepare Dataset

First, we want to define our prediction area (Costa Rica) and pass it to GEE. In order to move a vector to GEE, we'll use the `ee.Geometry.*` module. We'll also define a `map_display()` function for displaying a map on screen.

In [7]:
def map_display(center, eeg_dict, tiles="OpensTreetMap", zoom=10):
    '''
    param center: Center of the map (Latitude and Longitude).
    param eeg_dict: Earth Engine Geometries or Tiles dictionary.
    param tiles: Mapbox Bright, Mapbox Control Room, Stamen Terrain, Stamen Toner, stamenwatercolor, cartodbpositron.
    param zoom: Initial zoom level for the map.

    return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center, tiles=tiles, zoom_start=zoom)
    for k,v in eeg_dict.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = EE_TILES.format(**v),
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [ ]:
# prediction area
# TODO: update these coords
xmin, ymin, xmax, ymax = [-72.778645, -16.651663, -72.66865, -16.57553]

# passing the prediction area to Earth Engine
costa_rica = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax])
center = costa_rica.centroid().getInfo()['coordinates']
center.reverse()
map_display(center, {'Costa Rica':costa_rica.getInfo()}, zoom=12)

Next, we'll read in the train/test dataset and create a visualization.

- train dataset (2800 points):

  - 1400 labeled as "pineapple"
  - 1400 labeled as "non-pineapple"

- test dataset (700 points):

  - 350 labeled as "pineapple"
  - 350 labeled as "non-pineapple"

In [ ]:
# import the train/test dataset
# TODO: update these filepaths, and divvy into train/test sets
train_pineapple = ee.FeatureCollection('users/csaybar/DLdemos/train_set')
test_pineapple = ee.FeatureCollection('users/csaybar/DLdemos/test_set')

"""
background_2018 = ee.FeatureCollection('users/sagems/background_2018')
pineapple_2018 = ee.FeatureCollection('users/sagems/pineapple_2018')
background_2019 = ee.FeatureCollection('users/sagems/background_2019')
pineapple_2019 = ee.FeatureCollection('users/sagems/pineapple_2019')
borders = ee.FeatureCollection('FAO/GAUL/2015/level0')
costa_rica = borders.filter(ee.Filter.eq('ADM0_NAME', 'Costa Rica'))
"""

# display the train/test dataset
db_crop = train_pineapple.merge(test_pineapple)
center = db_crop.geometry().centroid().getInfo()['coordinates']
center.reverse()

eeg_dict = {'train': train_pineapple.draw(**{'color': 'FF0000', 'strokeWidth': 5}).getMapId(),
            'test' : test_pineapple.draw(**{'color': '0000FF', 'strokeWidth': 5}).getMapId(),
            'CostaRica': costa_rica.getInfo()
            }

map_display(center, eeg_dict, zoom=8)

For training the model, we will use the Harmonized Sentinel 2 dataset with images rendered at a 30-meter spatial resolution.

In [ ]:
from collections import OrderedDict

# load the dataset
# TODO: update these filepaths
s2 = ee.ImageCollection('users/csaybar/GLC30PERU').max().eq(10).rename('target')

# vizualize the dataset
s2_id = s2.getMapId()
eeg_dict['Sentinel2'] = s2_id

# change the order of the dictionary
key_order = ['Sentinel2', 'CostaRica', 'train', 'test']
eeg_dict = OrderedDict((k, eeg_dict[k]) for k in key_order)

map_display(center, eeg_dict, zoom=8)

Now, we're going to obtain the input data for mapping pineapple plantation areas from the Copernicus sensor (???). Also, we're going to define a function to mask cloud-cover in pixels.

The cloud-masking function from the tutorial is included as `maskS2clouds_tutorial()`.

In [13]:
def maskS2clouds(image):
    '''
    param image: Image input.

    return: Cloud-masked image.
    '''
    qa = image.select('QA10')

    # bits 10 and 11 are clouds and cirrus, respectively
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # both flags should be set to zero, indicating clear conditions
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000)

In [14]:
def maskS2clouds_tutorial(image):
  '''
  Function to mask clouds based on the pixel_qa band of Landsat 5 data. See:
  https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LT05_C01_T1_SR

  param img: image input Landsat 5 SR image

  return: cloud-masked Landsat 5 image
  '''
  qa = image.select('pixel_qa')
  cloud = qa.bitwiseAnd(1 << 5)\
            .And(qa.bitwiseAnd(1 << 7))\
            .Or(qa.bitwiseAnd(1 << 3))
  mask2 = image.mask().reduce(ee.Reducer.min())
  return image.updateMask(cloud.Not()).updateMask(mask2)


Now we will filter and reduce the entire Sentinel-2 dataset, considering the following:

- Select only bands [R, G, B, NIR].

- Filter by cloud pixel coverage by scene (< 20%).

- Filter by date (the tutorial selects 1 year)

- Apply `mask2cloud()` function to each image.

- Get the median of the ImageCollection.

- Clip the image considering the study area.

In [ ]:
# prepare the satellite image (Sentinel-2)
RGB_bands = ['B3','B2','B1']  # RGB
NDVI_bands = ['B4','B3']      # NIR

# NOTE: tutorial has 2 different satellite sensor source things, but I'm under the impression we're
#       using Sentinel-2 for both, just at different times? idk what's going on here lol
# TODO: change this to our stuff
s2_cop = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")\
               .filterBounds(db_crop)\
               .filterDate('2005-01-01', '2006-12-31')\
               .filter(ee.Filter.lt('CLOUD_COVER', 20))\
               .map(maskS2clouds)\
               .median()\
               .multiply(0.0001)

s2_cop_ndvi = s2_cop.normalizedDifference(NDVI_bands).rename(['NDVI'])
s2_cop_rgb = s2_cop.select(RGB_bands).rename(['R','G','B'])
s2_cop = s2_cop_rgb.addBands(s2_cop_ndvi).addBands(s2_cop)

In [ ]:
from collections import OrderedDict

# create a visualization with Folium
visParams_s2_cop = {
  'bands': ['R', 'G', 'B'],
  'min': 0,
  'max': 0.5,
  'gamma': 1.4,
}

s2_cop_mapID = s2.getMapId(visParams_s2_cop)
eeg_dict['Sentinel2_Copernicus'] = s2_cop_mapID

# change the order of the dictionary
key_order = ['Sentinel2_Copernicus', 'Sentinel2', 'CostaRica', 'train', 'test']
eeg_dict = OrderedDict((k, eeg_dict[k]) for k in key_order)

map_display(center, eeg_dict, zoom=8)

The key to success for integrating GEE with a CNN pipeline is the `ee.Image.neighborhoodToArray()` function. It turns the neighborhood of each pixel in a scalar image into a 2D array (see image below). Axes 0 and 1 of the output array correspond to Y and X axes of the image, respectively. The output image will have as many bands as the input; each output band has the same mask as the corresponding input band. The footprint and metadata of the input image are preserved.

There are several restrictions about the max number of features that a table can save (~ 10 M). For this reason, the `saveCNN_batch()` function is created as below:

In [ ]:
import numpy as np
import time

def saveCNN_batch(image, point, kernel_size, scale, file_prefix, selectors, folder, bucket='bag_csaybar'):
  """
    Export a dataset for semantic segmentation by batches

  Params:
  ------
    - image : ee.Image to get pixels from; must be scalar-valued.
    - point : Points to sample over.
    - kernel_size : The kernel specifying the shape of the neighborhood. Only fixed, square and rectangle kernels are supported.
      Weights are ignored; only the shape of the kernel is used.
    - scale : A nominal scale in meters of the projection to work in.
    - file_prefix : Cloud Storage object name prefix for the export.
    - selector : Specified the properties to save.
    - bucket : The name of a Cloud Storage bucket for the export.
  """
  print('Found Cloud Storage bucket.' if tf.io.gfile.exists('gs://' + bucket)
    else 'Output Cloud Storage bucket does not exist.')

  # download the points (Server -> Client)
  nbands = len(selectors)
  points = train_pineapple.geometry().getInfo()['coordinates']
  nfeatures = kernel_size*kernel_size*nbands*len(points) #estimate the totals # of features

  image_neighborhood = image.neighborhoodToArray(ee.Kernel.rectangle(kernel_size, kernel_size, 'pixels'))
  filenames = []

  # threshold considering the max number of features permitted to export
  if nfeatures > 3e6:
    nparts = int(np.ceil(nfeatures/3e6))
    print('Dataset too long, splitting it into '+ str(nparts),'equal parts.')

    nppoints = np.array(points)
    np.random.shuffle(nppoints)

    count_batch = 1

    for batch_arr in np.array_split(nppoints,nparts):

      fcp = ee.FeatureCollection([
          ee.Feature(ee.Geometry.Point(p),{'class':'NA'})
          for p in batch_arr.tolist()
      ])

      # pineapple dataset (fcp-points) collocation to each S2 grid cell value
      train_db = image_neighborhood.sampleRegions(collection=fcp, scale=scale)
      filename = '%s/%s-%04d_' % (folder,file_prefix,count_batch)

      # create the tasks for passing of GEE to Google storage
      print('sending the task #%04d'%count_batch)
      Task = ee.batch.Export.table.toCloudStorage(
        collection=train_db,
        selectors=selectors,
        description='Export batch '+str(count_batch),
        fileNamePrefix=filename,
        bucket=bucket,
        fileFormat='TFRecord')

      Task.start()
      filenames.append(filename)
      count_batch+=1

      while Task.active():
        print('Polling for task (id: {}).'.format(Task.id))
        time.sleep(3)

    return filenames

  else:
    train_db = image_neighborhood.sampleRegions(collection=points, scale=scale)
    Task = ee.batch.Export.table.toCloudStorage(
      collection=train_db,
      selectors=selectors,
      description='Training Export',
      fileNamePrefix=file_prefix,
      bucket=bucket,
      fileFormat='TFRecord')
    Task.start()

    while Task.active():
      print('Polling for task (id: {}).'.format(Task.id))
      time.sleep(3)

    return file_prefix

Unfortunately, you cannot use Tensorflow directly in Earth Engine. To overcome this limitation, the function `saveCNN_batch()` use Google Cloud Storage Bucket (you could use Google Drive instead) to save the dataset since both GEE and Tensorflow can access to it.

In [ ]:
selectors = ['R','G','B','NDVI','target']

# TODO: I think the folder and bucket params, at the very least, need to be changed
train_filenames = saveCNN_batch(s2_cop, train_pineapple, 128, 30, 'trainUNET', selectors, folder='unet', bucket='csaybar')
test_filenames = saveCNN_batch(s2_cop, test_pineapple, 128, 30, 'testUNET', selectors, folder='unet', bucket='csaybar')

### Create a TensorFlow Dataset

Read data from the TFRecord file into a tf.data.Dataset. Pre-process the dataset to get it into a suitable format for input into the DNN model.

In [ ]:
def input_fn(fileNames, num_epochs=None, shuffle=True, batch_size=16, side = 257):
  # read `TFRecordDatasets`
  dataset = tf.data.TFRecordDataset(fileNames, compression_type='GZIP')

  # names of the features
  feature_columns = {
    'R': tf.io.FixedLenFeature([side, side], dtype=tf.float32),
    'G': tf.io.FixedLenFeature([side, side], dtype=tf.float32),
    'B': tf.io.FixedLenFeature([side, side], dtype=tf.float32),
    'NDVI': tf.io.FixedLenFeature([side, side], dtype=tf.float32),
    'target': tf.io.FixedLenFeature([side, side], dtype=tf.float32)
  }

  # parsing function
  def parse(example_proto):
    parsed_features = tf.io.parse_single_example(example_proto, feature_columns)
    parsed_features = {key:value[1:side,1:side] for key,value in parsed_features.items()}

    # separate the class labels from the training features
    labels = parsed_features.pop('target')
    return parsed_features, tf.cast(labels, tf.int32)

  # convert FeatureColumns to a 4D tensor
  def stack_images(features,label):
    nfeat = tf.transpose(tf.squeeze(tf.stack(list(features.values()))))
    nlabel = (tf.transpose(label))[:,:,tf.newaxis]
    return nfeat, nlabel

  dataset = dataset.map(parse, num_parallel_calls=4)
  dataset = dataset.map(stack_images, num_parallel_calls=4)

  if shuffle:
    dataset = dataset.shuffle(buffer_size = batch_size * 10)

  dataset = dataset.batch(batch_size)
  dataset = dataset.repeat(num_epochs)

  return dataset

In [ ]:
# TODO: update these filepaths
folder = 'unet'
bucket = 'bag_csaybar'

files_list = !gsutil ls 'gs://'{bucket}'/'{folder}

train_file_prefix = 'trainUNET'
train_file_path = [s for s in files_list if train_file_prefix in s]

test_file_prefix = 'testUNET'
test_file_path = [s for s in files_list if test_file_prefix in s]

In [ ]:
train_dba = input_fn(train_file_path, num_epochs=100, shuffle=True, batch_size=3)
test_dba = input_fn(test_file_path, num_epochs=1, shuffle=False, batch_size=1)

### Visualize Patches
Visualize some of the map patches in the dataset.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
display_num = 5
plt.figure(figsize=(14, 21))

c = 0
for i in range(1, display_num):
  for x in test_dba.take(i):
    x

  tensor = tf.squeeze(x[0]).numpy()[:,:,[3,1,0]]
  target = tf.squeeze(x[1])

  # print(target.sum())
  plt.subplot(display_num, 2, c + 1)
  plt.imshow(tensor)
  plt.title("RGB SENTINEL-2")

  plt.subplot(display_num, 2, c + 2)
  plt.imshow(target)
  plt.title("Pineapple Plantations")
  c += 2

plt.show()

### Set-Up Params
We'll begin by defining some constant parameters.

In [ ]:
IMG_SHAPE = (256, 256, 4)
EPOCHS = 10

### Create U-Net Model

Here, we'll create a convolutional neural network model with:
- 5 encoder layers.
- 5 decoder layer.
- 1 output layer.

The **encoder layer** is composed of a linear stack of Conv, BatchNorm, and Relu operations followed by a MaxPool. Each MaxPool will reduce the spatial resolution of our feature map by a factor of 2. We keep track of the outputs of each block as we feed these high-resolution feature maps with the decoder portion.

The **decoder layer** is comprised of UpSampling2D, Conv, BatchNorm, and Relu. Note that we concatenate the feature map of the same size on the decoder side.

Finally, for the **output layer**, we add a Conv operation that performs a convolution along the channels for each individual pixel (kernel size of (1, 1)) that outputs our final segmentation mask in grayscale.

Additionally, early stopping, TensorBoard, and best-model callback are utilized.

In [ ]:
from tensorflow.keras import layers

def conv_block(input_tensor, num_filters):
  encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
  encoder = layers.BatchNormalization()(encoder)
  encoder = layers.Activation('relu')(encoder)
  encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
  encoder = layers.BatchNormalization()(encoder)
  encoder = layers.Activation('relu')(encoder)
  return encoder

def encoder_block(input_tensor, num_filters):
  encoder = conv_block(input_tensor, num_filters)
  encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)

  return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
  decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
  decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
  decoder = layers.BatchNormalization()(decoder)
  decoder = layers.Activation('relu')(decoder)
  decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
  decoder = layers.BatchNormalization()(decoder)
  decoder = layers.Activation('relu')(decoder)
  decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
  decoder = layers.BatchNormalization()(decoder)
  decoder = layers.Activation('relu')(decoder)
  return decoder

inputs = layers.Input(shape=IMG_SHAPE)
# 256
encoder0_pool, encoder0 = encoder_block(inputs, 32)
# 128
encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64)
# 64
encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128)
# 32
encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256)
# 16
encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512)
# 8
center = conv_block(encoder4_pool, 1024)
# center
decoder4 = decoder_block(center, encoder4, 512)
# 16
decoder3 = decoder_block(decoder4, encoder3, 256)
# 32
decoder2 = decoder_block(decoder3, encoder2, 128)
# 64
decoder1 = decoder_block(decoder2, encoder1, 64)
# 128
decoder0 = decoder_block(decoder1, encoder0, 32)
# 256

outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(decoder0)

### Define the Model

We'll define the model by specifying the associated inputs and outputs.

In [ ]:
from tensorflow.keras import models
model = models.Model(inputs=[inputs], outputs=[outputs])
model.summary()

### Define Loss Function

Defining loss and metric functions are simple with Keras. Just define a function that takes both the True and Predicted labels for a given example. Dice loss is a metric that measures overlap. We use dice loss here because it performs better with class imbalanced problems by design. Using cross entropy (another loss function) is more of a proxy which is easier to maximize. Instead, we maximize our objective directly.

In [ ]:
from tensorflow.keras import losses

def dice_coeff(y_true, y_pred):
    smooth = 1.
    # flatten
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def bce_dice_loss(y_true, y_pred):
  loss = losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
  return loss

### Compile the Model

We'll use our custom loss function to minimize. In addition, we specify what metrics we want to keep track of as we train. Note that metrics are not actually used during the training process to tune the parameters, but are instead used to measure performance of the training process.

In [ ]:
model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[dice_loss])

### Train the Model

Training your model with tf.data involves simply providing the model's fit function with your training/validation dataset, the number of steps, and epochs.

We also include a Model callback, `ModelCheckpoint`, that will save the model to disk after each epoch. We configure it so that it only saves our highest performing model. Note that saving the model captures more than just the weights of the model: by default, it saves the model architecture, weights, as well as information about the training process such as the state of the optimizer, etc.

In [ ]:
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import datetime

# callbacks time
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# early stopping
es = EarlyStopping(monitor='val_loss', patience=10)

# model checkpoint
mcp = ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
n_train = 550
batch_size = 3

# train the model (tutorial stops after 15min train time)
history = model.fit(train_dba,
                    steps_per_epoch=int(np.ceil(n_train / float(batch_size))),
                    epochs=EPOCHS,
                    validation_data=test_dba,
                    callbacks=[tensorboard_callback,es,mcp])

In [ ]:
%tensorboard --logdir logs
#!kill 607

### Evaluate the Model

In [ ]:
model.evaluate(x=test_dba)

### Prediction

In [ ]:
# TODO: update these filepaths with 2020-23 data
s2_cop = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")\
               .filterBounds(costa_rica)\
               .filterDate('2005-01-01', '2006-12-31')\
               .filter(ee.Filter.lt('CLOUD_COVER', 20))\
               .map(maskS2clouds)\
               .median()\
               .multiply(0.0001)

s2_cop_ndvi = s2_cop.normalizedDifference(NDVI_bands).rename(['NDVI'])
s2_cop_rgb = s2_cop.select(RGB_bands).rename(['R','G','B'])
s2_cop = s2_cop_rgb.addBands(s2_cop_ndvi)

In [ ]:
from collections import OrderedDict

# vizualize the dataset
s2_cop_id = s2_cop.clip(costa_rica.buffer(2500)).getMapId({'max':0.6,'min':0})
center = costa_rica.centroid().getInfo()['coordinates']
center.reverse()
map_display(center, {'s2_cop_id':l5id}, zoom=11)

To export the results to Google Cloud Storage, it's best to define the following format options parameters in order to save memory:

- **patchDimensions**: Patch dimensions tiled over the export area, covering every pixel in the bounding box exactly once (except when the patch dimensions do not evenly divide the bounding box in which case the lower and right sides are trimmed).

- **compressed**: If true, compresses the .tfrecord files with gzip and appends the ".gz" suffix.

In [ ]:
# TODO: update these filepaths
output_bucket = 'bag_csaybar'
image_file_prefix = 'unet/Predict_CamanaValleyCrop'

# specify file dimensions
image_export_format_options = {
  'patchDimensions': [256, 256],
  'compressed': True
}

# set-up the task
image_task = ee.batch.Export.image.toCloudStorage(
  image=l5,
  description='Image Export',
  fileNamePrefix=image_file_prefix,
  bucket=output_bucket,
  scale=30,
  fileFormat='TFRecord',
  region=Camana_valley.buffer(2500).getInfo()['coordinates'],
  formatOptions=image_export_format_options,
)

# start the export
image_task.start()

In [ ]:
import time
while image_task.active():
  print('Polling for task (id: {}).'.format(imageTask.id))
  time.sleep(5)

Now it's time to classify the image that was exported from GEE to GCS using Tensorflow. If the exported image is large, it will be split into multiple TFRecord files in its destination folder. There will also be a JSON sidecar file called "the mixer" that describes the format and georeferencing of the image. Here, we will find the image files and the mixer file, getting some info out of the mixer that will be useful during model inference.

In [ ]:
files_list = !gsutil ls 'gs://'{output_bucket}'/unet/'
export_files_list = [s for s in files_list if image_file_prefix in s]

# get the list of image files and JSON mixer file
image_files_list = []
json_file = None
for f in export_files_list:
  if f.endswith('.tfrecord.gz'):
    image_files_list.append(f)
  elif f.endswith('.json'):
    json_file = f

# ensure files are in correct order
print(json_file)

The mixer contains metadata and georeferencing information for the exported patches, each of which is in a different file. Read the mixer to get some information needed for prediction.

In [ ]:
import json
from pprint import pprint

# load the contents of the mixer file to a JSON object
jsonText = !gsutil cat {jsonFile}

# get a single string w/ newlines from the IPython.utils.text.SList
mixer = json.loads(jsonText.nlstr)
pprint(mixer)

The next function is slightly different from the to the previously defined `input_fn()`. Mainly, this is because the pixels are written into records as patches, and we need to read the patches in as one big tensor (one patch for each band), and then flatten them into lots of little tensors. Once the `predict_input_fn()` is defined, it can handle the shape of the image data, so all you need to do is feed it directly to the trained model to make predictions.

In [ ]:
def predict_input_fn(file_names, side, bands):

  # read `TFRecordDatasets`
  dataset = tf.data.TFRecordDataset(file_names, compression_type='GZIP')

  features_dict = {x:tf.io.FixedLenFeature([side, side], dtype=tf.float32) for x in bands}

  # parsing function
  def parse_image(example_proto):
    parsed_features = tf.io.parse_single_example(example_proto, features_dict)
    return parsed_features

  def stack_images(features):
    nfeat = tf.transpose(tf.squeeze(tf.stack(list(features.values()))))
    return nfeat

  dataset = dataset.map(parse_image, num_parallel_calls=4)
  dataset = dataset.map(stack_images, num_parallel_calls=4)
  dataset = dataset.batch(side*side)

  return dataset

In [ ]:
predict_db = predict_input_fn(fileNames=image_files_list, side=256, bands=['R', 'G', 'B', 'NDVI'])
predictions = model.predict(predict_db)

Now that there's a `np.array` of probabilities in "predictions", it's time to write them back into a file. You will write directly from TensorFlow to a file in the output Cloud Storage bucket.

Iterate over the list and write the probabilities in patches. Specifically, we need to write the pixels into the file as patches in the same order they came out. The records are written as serialized `tf.train.Example` protos. This might take a while.

In [ ]:
# instantiate the writer
# TODO: change filepaths
PATCH_WIDTH , PATCH_HEIGHT = [256,256]
output_image_file = 'gs://' + output_bucket + '/unet/CamanaValleyCrop.TFRecord'
writer = tf.io.TFRecordWriter(output_image_file)

# Every patch-worth of predictions we'll dump an example into the output
# file with a single feature that holds our predictions. Since our predictions
# are already in the order of the exported data, the patches we create here
# will also be in the right order.

curPatch = 1
for  prediction in predictions:
  patch = prediction.squeeze().T.flatten().tolist()

  if (len(patch) == PATCH_WIDTH * PATCH_HEIGHT):
    print('Done with patch ' + str(curPatch) + '...')
    # Create an example
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'crop_prob': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch))
        }
      )
    )

    writer.write(example.SerializeToString())
    curPatch += 1

writer.close()

### Upload Predictions to EEG

At this stage, there should be a predictions TFRecord file sitting in the output Cloud Storage bucket. Use the gsutil command to verify that the predictions image (and associated mixer JSON) exist and have non-zero size.

In [ ]:
!gsutil ls -l {output_image_file}


Upload the image to Earth Engine directly from the Cloud Storage bucket with the earthengine command. Provide both the image TFRecord file and the JSON file as arguments to earthengine upload.

In [ ]:
# TODO: update username
# TODO: update filepaths
USERNAME = 'csaybar'
output_asset_ID = 'users/' + USERNAME + '/CamanaCrop_UNET'
print('Writing to ' + output_asset_ID)

In [ ]:
# start the upload (it might take a while)
!earthengine upload image --asset_id={output_asset_ID} {output_image_file} {json_file}

### Display Results

Display the results using Folium!

In [ ]:
probs_image = ee.Image(output_asset_ID)
predictions_image = ee.Image(output_asset_ID).gte(0.500)
eeg_dict = {'PineappleProbability':probs_image.getMapId({'min':0.49,'max':0.498}),
            'Pineapple':predictions_image.getMapId()}

center = costa_rica.centroid().getInfo()['coordinates']
center.reverse()

map_display(center, eeg_dict, zoom=13)

All done!!!